# Make 24 Hr Coding Sheet

This file is used for compiling the 24 hour recorded coding sheets into one excel file per rat. 

#### *IMPORTANT:*
After you create the new excel file. You must move it out of the folder this file is in before running this file again. The way this file works is that it will automatically combine every excel file in the folder this file is in and in the subfolders of the folder this file is in. If you do not remove the newly created compiled 24 hr file before running this file again, an error will be raised or the new file will be added with the hour files which would be a problem.

Make sure all files that are being compiled are error free or running this file will result in an error.

In [1]:
#import necessary modules
import pandas as pd
import numpy as np
import os
import datetime

#extraction definition
def extract(data):
    #read excel file name
    df = pd.read_excel(data)
    #make column from 'Observation id'
    df_t = np.array(df['Observation id'])
    #make that column a list
    df_l = list(df_t)
    #make a dataframe from the rows below the 'Time' 
    df_v = df.iloc[df_l.index('Time')+1:,:]
    #make the columns name the same names as the ones in the 'Time' row
    df_v.columns = df.iloc[df_l.index('Time')]
    #drop 'Media file path' and 'Behavioral category'
    df_d = df_v.drop(['Media file path', 'Behavioral category'], axis = 1)
    #set the subject to index
    df_i = df_d.set_index('Subject')
    return df_i

In [2]:
#find where the files combining from
path = os.getcwd()
files = os.listdir(path)

#get the files that end with '.xlsx' and '.xls'
xlsxfiles = [os.path.join(root, name)
             for root, dirs, files in os.walk(path)
             for name in files
             if name.endswith((".xlsx", ".xls"))]

In [3]:
#extract the dataframes from the excel files
#make an empty array
extracted_dataframes = []
for i in range(len(xlsxfiles)):
    #loop through xlsxfiles names and extract the data
    data = extract(xlsxfiles[i])
    #add that to empty array
    extracted_dataframes.append(data)
#combine the dataframes to one dataframe
all_hours = pd.concat(extracted_dataframes[:], sort=True)

#make column that has the hour the data was taken from the rat name and hour
#make empty set
hours = []
#set the 'Subject' column (index column) as an array to 'subjects'
subjects = np.array(all_hours.index)
#loop through subjects
for i in range(len(subjects)):
    #make each name into a string
    strings = str(subjects[i])
    #select the last two characters (for the hour)
    last2 = strings[-2:]
    #add them to the empty array
    hours.append(last2)
#set the hours as column 'Hour'
all_hours['Hour']=hours

#make column that has the name the data was taken from the rat name and hour
#make empty set
name=[]
#loop through subjects
for i in range(len(subjects)):
    #make each name into a string
    strings = str(subjects[i])
    #select the first five characters (for the hour)
    namer = strings[:5]
    #add them to the empty array
    name.append(namer) 
#set the array name as column 'Name'
all_hours['Name']=name

#set times equal to the Time column
times = all_hours.Time
#convert times to floats
times = list(map(float, times))
#create an array from times
nptimes = np.array(times)

#set hour_val to column 'Hour'
hour_val = all_hours.Hour
#convert hour times to intergers
hour_val = list(map(int, hour_val))
#make hour_val into an array
np_hour_val = np.array(hour_val)
#add nptimes and 3600 x np_hour_val to get total seconds from beginning of day
npact_time = nptimes + 3600*np_hour_val
#set the total seconds, npact_time, to column 'seconds'
all_hours['seconds'] = npact_time

#make empty array
date_time = []
#import modules
import datetime
from datetime import timedelta
#loop through npact_time
for i in range(len(npact_time)):
    #use datetime.datetime.fromtimestamp to convert seconds into datetime and add 5 hour so it starts on 1970-1-1 0:00
    timez = datetime.datetime.fromtimestamp(npact_time[i])
    #add the datetime to empty set
    date_time.append(timez)
#set the datetime to column 'Datetime'
all_hours['Datetime']=date_time

#make empty array
act_time = []
#loop through npact_time
for i in range(len(npact_time)):
    #use datetime.timedelta to get difference from 0:00 and the seconds
    realtime = datetime.timedelta(seconds = npact_time[i])  + timedelta(hours=5)
    #add the difference to empty set
    act_time.append(str(realtime))
#set the difference in time to column 'Act_Time'
all_hours['Act_Time']=act_time

#divide npact_time by 3600s to get hours in array
nptime_hr = npact_time/3600
#make column 'Real_Time_Hr' equal to nptime_hr
all_hours.loc[:,'Real_Time_Hr']=nptime_hr

#set df to all_hours
df=all_hours

#set the index to the datetime by converting the column seconds into datetime with unit of seconds
df = df.reset_index()
df.index = pd.to_datetime(df.seconds, unit='s')
df = df[['Subject','Behavior', 'Status', 'Time', 'Hour', 'Name', 'seconds', 'Datetime','Act_Time', 'Real_Time_Hr']]
df.head(5)

,Subject,Behavior,Status,Time,Hour,Name,seconds,Datetime,Act_Time,Real_Time_Hr
seconds,,,,,,,,,,
1970-01-01 00:00:00.109,Rat02_00,Sleeping/Resting,START,0.109,00,Rat02,0.109,1969-12-31 19:00:00.109,5:00:00.109000,0.000030
1970-01-01 00:01:05.251,Rat02_00,Sleeping/Resting,STOP,65.251,00,Rat02,65.251,1969-12-31 19:01:05.251,5:01:05.251000,0.018125
1970-01-01 00:02:48.002,Rat02_00,Water,START,168.002,00,Rat02,168.002,1969-12-31 19:02:48.002,5:02:48.002000,0.046667
1970-01-01 00:03:32.151,Rat02_00,Water,STOP,212.151,00,Rat02,212.151,1969-12-31 19:03:32.151,5:03:32.151000,0.058931
1970-01-01 00:03:42.901,Rat02_00,Rearing,START,222.901,00,Rat02,222.901,1969-12-31 19:03:42.901,5:03:42.901000,0.061917


The code below will automatically make a separate 24 hr excel file for each rat. If an hour is not included for a certain rat, then you should manually change the name of that excel file to reflect that. For example, rat 26 is missing hour 19. The following code will make a excel file for rat 26 called "Rat26_full_day.xlsx". You should change it to "Rat26_full_day_except_hr_19.xlsx".

In [4]:
#loop through the unique names of the df.Name column
for i in np.unique(df.Name):
    dfname = df[df.Name == i]
    dfname.to_excel(i + '_full_day.xlsx')

##### To make just one new 24 hr excel file for a new rat use the following code.

Manually replace the '##' with the rat number of the desired rat and delete the hashtags at the beginning of the lines.

An example would look like:

dfrat = df[df.Name == 'Rat37']

dfrat.to_excel('Rat37_full_day.xlsx')

In [5]:
#dfrat = df[df.Name == 'Rat##']
#dfrat.to_excel('Rat##_full_day.xlsx')